# 2. Data Preparation

본 챕터에서는 모빌리티 시뮬레이션을 구축하는데 필요한 기본적인 입력데이터들을 살펴본다. 단순히 데이터를 살펴보는데서 끝나지 않고 간단한 데이터의 전처리 및 시각화의 실습과정도 포함하고 있다.

매우 다양한 형태의 모빌리티 데이터가 존재하며, 이들을 분류하는 기준 역시 분석자의 목표에 따라 여려가지로 구분할 수 있지만 본 책에서는 크게 아래와 같은 형태의 데이터를 살펴볼 것이다.
1. 통행수요 데이터 (Travel Demand)
    - 시뮬레이션을 하기 위한 승객의 수요를 생성하는 기초자료로 활용됨
2. 통행시간 데이터 (Travel Time)
    - 시뮬레이션을 Calibration 하는데 활용
    - e.g., 출퇴근 시간의 교통 정체, 새벽시간대의 빠른 속도를 시뮬레이션에 반영 가능함
3. 도로 네트워크 데이터 (Road Network)
    - 도로 위를 움직이는 차량 구현 (Vehicle Router)
4. 대중교통 네트워크 데이터 (Public Transit Network)
    - 버스, 지하철과 같은 노선과 배차간격이 정해진 모빌리티 시스템을 구현하는데 사용

## 2.1 Travel Demand Data (Trip record data)

### 2.1.1 활용 데이터
- 통행수요는 특정지역에서 출발(Trip Production)하거나, 특정지역으로 도착하는 통행(Trip Attraction) 형태로 존재할 수도 있고, 혹은 더 세분화 한다면 출발지-목적지 단위의 통행수요(Origin-Destination (OD) Travel Demand)로 나타낼 수 있다. 
- 최근 많은 사람들이 스마트폰을 사용하고 따라서 통신사 기지국 기반으로 세밀한 통행패턴이 수집 가능하며, O-D 단위의 통행수요 데이터를 구득하기가 수월해졌다. 따라서 본 교재에서는 O-D 단위의 통행 데이터를 주로 활용하는 실습을 진행할 예정이다. 
- 본 예제에서는 서울시에서 제공하는 수도권 생활이동 데이터를 활용하였다. 
    - https://data.seoul.go.kr/dataVisual/seoul/capitalRegionLivingMigration.do
- 컬럼의 명세는 아래와 같다. 행정동 단위의 O-D 통행량 및 통행시간 정보를 제공해주고 있다. 더 상세하게는 내/외국인 구분, 국적, 이동목적과 같은 정보도 제공을 해준다.


| 순번 | 영문 컬럼명 | 컬럼 설명 | NULL 여부 | NULL 대체값 | 형식 | 규칙 | 데이터 허용범위 | 비고 |
|------|------------|-----------|-----------|-------------|------|------|-----------------|------|
| 1 | O_ADMDONG_CD | 출발 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 2 | D_ADMDONG_CD | 도착 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 3 | ST_TIME_CD | 출발 시간 | X | - | STRING | - | - | 7~9시/17시~19시는 20분단위, 그 외 1시간 단위 |
| 4 | FNS_TIME_CD | 도착 시간 | X | - | STRING | - | - | 7~9시/17시~19시는 20분단위, 그 외 1시간 단위 |
| 5 | IN_FORN_DIV_NM | 내/외국인 구분 | X | - | STRING | - | - | 내국인, 단기외국인, 장기외국인 |
| 6 | FORN_CITIZ_NM | 국적 | X | - | STRING | - | - | 　 |
| 7 | MOVE_PURPOSE | 이동 목적 | X | - | STRING | - | - | 1: 출근, 2 : 등교, 3: 귀가, 4: 쇼핑, 5: 관광, 6: 병원, 7: 기타 |
| 8 | MOVE_DIST | 평균 이동 거리(m) | X | - | DOUBLE | - | - | 　 |
| 9 | MOVE_TIME | 평균 이동 시간(분) | X | - | DOUBLE | - | - | 　 |
| 10 | CNT | 이동인구 수 | X | - | DOUBLE | (18,2) | - | 　 |
| 11 | ETL_YMD | 기준 년월 일 | X | - | STRING | yyyyMMdd | 데이터 기준 당일 | - |

데이터셋 명: 출도착 행정동별 이동 목적 데이터  ·
DB 명: PURPOSE_ADMDONG3

### 2.1.2 데이터 읽기 및 간단한 전처리

- 간단한 예제를 위해 서울시에서 제공하는 [수도권 생활이동](https://data.seoul.go.kr/dataVisual/seoul/capitalRegionLivingMigration.do) 데이터 중, 2024년 3월 27일 하루치의 데이터만을 예제로 사용하였다. 
- 원활한 실습을 위해 모든 데이터는 Google Drive에 업로드 하였으며, `gdown` 패키지를 사용해 데이터를 다운로드 받은 후 로드하는 방식을 사용하였다. 

In [8]:
import gdown
import pandas as pd
import os

In [13]:
def download_and_read_parquet(file_id, output_path="../data/chp2_od_data.parquet"):
    try:
        # Google Drive에서 파일 다운로드
        gdown.download(id=file_id, output=output_path, quiet=False)
        
        # Parquet 파일을 DataFrame으로 읽기
        df = pd.read_parquet(output_path)
        
        # 임시 파일 삭제 (데이터 용량이 매우 큰 경우 사용)
        # os.remove(output_path) # 다운로드 받은 데이터를 삭제하고 싶지 않을 때는 해당 라인을 주석처리
        
        return df
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [14]:
# 파일 불러오기
file_id = "1teftc2KgWrG0Jn2SyyY3pipkBugmvnex" # 구글 드라이브에 업로드 된 파일의 ID
df = download_and_read_parquet(file_id)

Downloading...
From: https://drive.google.com/uc?id=1teftc2KgWrG0Jn2SyyY3pipkBugmvnex
To: f:\mobility-simulation-book\data\chp2_od_data.parquet
100%|██████████| 56.6M/56.6M [00:01<00:00, 35.2MB/s]


인터넷 환경으로 인해 파일을 불러오는게 느리다면, github에서 clone 할 때 가지고 온 data 폴더에 있는 파일을 직접 열 수 있다.  
아래 코드를 실행해 보자.  
```code-cell
df = pd.read_parquet("../data/chp2_od_data.parquet")
```

In [17]:
df

,O_ADMDONG_CD,D_ADMDONG_CD,ST_TIME_CD,CNT,MOVE_DIST,MOVE_TIME
0,11110515,11110515,0,25.54,151.875714,1683.637143
1,11110515,11110515,1,79.29,104.120000,1210.596667
2,11110515,11110515,2,103.07,175.205714,360.190000
3,11110515,11110515,3,16.30,239.946667,2441.596667
4,11110515,11110515,4,34.73,215.720000,860.990000
...,...,...,...,...,...,...
3792913,52800410,41590360,11,2.56,69512.350000,4087.830000
3792914,52800420,11350570,14,3.50,66976.630000,6354.770000
3792915,52800420,11440630,9,3.48,63894.260000,3725.740000
3792916,52800420,41273610,17,2.33,69839.840000,1864.100000


## 2.2 Travel Time Data

## 2.3 Road Network and Geometry

### 2.3.1 Open Street Map

[오픈스트리트맵 한국](https://osm.kr/usage/)에서는 대한민국의 OSM을 이용하는 상세한 방법에 대한 메뉴얼을 제공해주고 있다. 상세한 내용은 이곳을 참고하면 되며, 본 장에서는 한국의 OSM 데이터를 불러와서 `OSMnx` 패키지로 간략히 분석하고 시각화 하는 작업을 진행한다.  

OSM을 이용해서 수도권 지역의 도로 데이터를 가져오고, 시각화 하는 작업을 수행  
OSMnx 이용해서 그래프 분석하고, 지난번 세미나 때 했던 간소화 하는 작업도 해볼 수 있지 않을까



## 2.4 GTFS (General Transit Feed Specification)

- 대중교통 네트워크 분석을 위한 데이터
- 향후 업데이트 예정

## 2.5 Exercise

GPT와 같은 AI 툴을 적극 활용해 문제를 풀어봅시다. 다만, 출력된 코드를 보고 이해하고, 코드가 실행되지 않는다면 어디서 실행되지 않는지 파악하고 올바르게 수정할 수 있어야 합니다.

### 2.5.1 통행 수요 데이터 분석 및 시각화

> 2.1 Section에서 불러온 2024년 3월 27일 통행 데이터를 가지고 아래와 같은 분석을 수행해보자.

1. 시간에 따른 통행량을 그래프로 시각화 해보자. 몇시에 통행이 가장 많으며, 그 양은 얼마인가?  
2. 공간에 따른 통행량의 분포를 읍면동 단위로 시각화 해보자. 어느 지역에서 출발 통행량(Trip Production)이 가장 많으며 어느 지역에서 도착 통행량 (Trip Attraction)이 가장 많은가?  
3. 통행의 시공간적 분포를 함께 분석해보자. 가장 효과적인 시각화 방안은 무엇일까?  